In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from experiment import experiment
from experiment import test_loss
import random
scaler = MinMaxScaler(feature_range=(-1, 1))
"""
#loading and sorting the data
diabetes_data = pd.read_csv("diabetes.csv").to_numpy()
diabetes_x = diabetes_data[:,:-1]
diabetes_y = diabetes_data[:,-1]
diabetes_y = np.where(diabetes_y == 0,-1, diabetes_y)
# avoid overflow error

diabetes_x = scaler.fit_transform(diabetes_x)
bias = np.ones((diabetes_x.shape[0],1))
diabetes_x = np.hstack((bias,diabetes_x))
"""
#loading sonar data
sonar_data = pd.read_csv("sonar.csv").to_numpy()
sonar_x = sonar_data[:,:-1]
sonar_x = scaler.fit_transform(sonar_x)
#add intercept
bias = np.ones((sonar_data.shape[0],1))
sonar_x = np.hstack((bias, sonar_x)).astype(float)
sonar_y = sonar_data[:,-1]
#convert labels to +1 -1
sonar_y = np.where(sonar_y == "M",1,-1)

bins = [1,2,3,4]

normal_iters, quant_iters,  w_quant, w = experiment(sonar_x,sonar_y, bins)

ValueError: zero-size array to reduction operation minimum which has no identity

In [3]:
from QuantLog import quantlogistic
from NormalLog import normallogistic

#generate random w to test gradient func on
w_test = np.random.rand(sonar_x.shape[1]) *2 -1

loss, ng = normallogistic(w_test,sonar_x.T,sonar_y.T)
loss, qg1 = quantlogistic(w_test,sonar_x.T,sonar_y.T,1)
loss, qg2 = quantlogistic(w_test,sonar_x.T,sonar_y.T,2)
loss, qg3 = quantlogistic(w_test,sonar_x.T,sonar_y.T,3)

gradients = np.hstack((ng,qg1,qg2,qg3))
print(gradients)

[[ 2.17657150e-01  3.09178744e-01  1.46537842e-01  1.13181504e-01]
 [-1.90048359e-01 -2.45258148e-01 -1.39353218e-01 -1.17343667e-01]
 [-1.93949742e-01 -2.64969530e-01 -1.38535319e-01 -1.13146599e-01]
 [-1.89941043e-01 -2.63621474e-01 -1.32003826e-01 -1.07111343e-01]
 [-1.98795515e-01 -2.76926682e-01 -1.40956520e-01 -1.13462086e-01]
 [-1.72988865e-01 -2.34635831e-01 -1.22469424e-01 -1.01026585e-01]
 [-1.23979420e-01 -1.76249956e-01 -8.67457668e-02 -7.13433483e-02]
 [-9.56895990e-02 -1.31268691e-01 -6.68921792e-02 -5.55611573e-02]
 [-1.32081032e-01 -1.64349211e-01 -9.40471942e-02 -8.13651661e-02]
 [-1.69398814e-01 -2.00474865e-01 -1.23174694e-01 -1.09065449e-01]
 [-1.63706273e-01 -1.87490458e-01 -1.23266814e-01 -1.09852137e-01]
 [-1.72435111e-01 -1.99317658e-01 -1.31165847e-01 -1.19436228e-01]
 [-1.45050688e-01 -1.67745553e-01 -1.10203072e-01 -1.01363515e-01]
 [-1.04167902e-01 -1.25740162e-01 -7.87070358e-02 -7.17237671e-02]
 [-1.04889280e-01 -1.46545647e-01 -7.07264245e-02 -5.74287025e